In [1]:
import robotic as ry
import numpy as np
import random
from bridgebuild_env import create_n_boxes

# this import the local manipulation.py .. to be integrated in robotic..
import manipulation as manip
import robot_execution as robex

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


### Task Env: Bridge Builder

In [12]:
C = ry.Config()
C.addFile(ry.raiPath("scenarios/pandaSingle.g"))
n_boxes = 3
keep_dist_between = ["box1", "box2", "box3"]
C = create_n_boxes(C=C, N=n_boxes, position="fixed")

# remove so collision with wrist camera is ignored
C.delFrame("panda_collCameraWrist")

# for convenience, a few definitions:
qHome = C.getJointState()
gripper = "l_gripper"
palm = "l_palm"
box1 = "box1"
box2 = "box2"
box3 = "box3"
table = "table"
boxSize = C.frame(box1).getSize()

C.view()

0

In [8]:
C.view()
bot = robex.Robot(C, on_real=False)
bot.goHome(C)

In [13]:
C.setJointState(qHome)
C.view_raise()
boxes_pos = [
    C.frame(box1).getPosition(),
    C.frame(box2).getPosition(),
    C.frame(box3).getPosition(),
]
boxes = [box1, box2, box3]
boxes_all = [box1, box2, box3]
n_success = 0
last_box_memory = []
while n_success <= 2:
    print(f"n_success: {n_success}")
    box = np.random.choice(boxes)
    not_picked_boxes = [i for i in boxes_all if i != box]

    qStart = C.getJointState()
    graspDirection = random.choice(["x", "y"])
    placeDirection = "z"
    info = f"placement {n_success}: grasp {graspDirection} place {placeDirection}"
    print("===", info)

    M = manip.ManipulationModelling(C, info, helpers=[gripper])
    M.setup_pick_and_place_waypoints(
        gripper, box, phases = 3., homing_scale=1e-1, accumulated_collisions=True
    )
    M.grasp_box(1.0, gripper, box, palm, graspDirection)
    print("place direction:", placeDirection)

    if n_success > 1:
        offset_xyz = [0.0, 0.0, 0.00]
        print(f"3rd Block: ({box})")
        dir_vec = (
            C.getFrame(last_box_memory[-2]).getPosition()
            - C.getFrame(last_box_memory[-1]).getPosition()
        )
        pos2 = C.getFrame(last_box_memory[-2]).getPosition()
        pos1 = C.getFrame(last_box_memory[-1]).getPosition()
        rel_pos = (pos1 + pos2) / 2 + np.array([0, 0, 0.06])
        dir_vec /= np.linalg.norm(dir_vec)

        M.komo.addObjective([2.0], ry.FS.position, [box], ry.OT.eq, [1e1], rel_pos)
        M.komo.addObjective([2.0], ry.FS.vectorZ, [box], ry.OT.eq, [1e1], dir_vec)

        print("Grasp Direction:", graspDirection)

    if n_success == 1:
        print(f"2nd Block: ({box})")
        M.place_box(2.0, box, table, palm, placeDirection)
        M.komo.addObjective(
            [2.0],
            ry.FS.negDistance,
            [box, last_box_memory[-1]],
            ry.OT.eq,
            [1e1],
            [-0.04],
        )
        M.komo.addObjective(
            [2.0],
            ry.FS.scalarProductXY,
            [box, last_box_memory[-1]],
            ry.OT.eq,
            [1e1],
            [0],
        )
        M.komo.addObjective(
            [2.0],
            ry.FS.positionRel,
            [box, last_box_memory[-1]],
            ry.OT.eq,
            [1.0],
            [0.0, 0.0, 0.1],
        )
        M.keep_dist_pairwise([], keep_dist_between, margin=0.08)
        #M.keep_distance([], box, last_box_memory[-1], margin = 0.06)


    M.keep_distance([], palm, table)
    # keep distance between boxes when placed defined by margin
    # x, y, z, smooth - table size: array([2.5 , 2.5 , 0.1 , 0.02])
    # place inital block as chosen by np.random.choice at a random location
    if n_success == 0:
        print(f"1st Block: ({box})")
        M.place_box(2.0, box, table, palm, placeDirection)
        initial_pos = [np.random.uniform(0.0, 0.1), np.random.uniform(0.0, 0.1)]

        M.target_relative_xy_position(
            2.0, box, table, initial_pos
        )
        M.keep_dist_pairwise([], keep_dist_between, margin=0.08)
        #M.keep_distance([], box, not_picked_boxes[0], margin = 0.06)
        #M.keep_distance([], box, not_picked_boxes[1], margin = 0.06)

    ways = M.solve()

    if not M.feasible:
        print("True")
        continue

    M2 = M.sub_motion(0)
    M2.keep_distance([0.3, 0.7], palm, box)

    M2.retract([0.0, 0.2], gripper, dist=0.012)
    M2.approach([0.8, 1.0], gripper)
    M2.solve()
    if not M2.feasible:
        continue

    M3 = M.sub_motion(1)
    # manip.ManipulationModelling(C, info)
    # M.setup_point_to_point_motion(ways[0], ways[1])
    M3.keep_distance([], table, box)


    M3.retract([0.0, 0.2], gripper, dist=0.025)
    M3.approach([0.8, 1.0], gripper)

    M3.solve()
    if not M3.ret.feasible:
        continue

    M2.play(C)
    C.attach(gripper, box)
    M3.play(C)
    C.attach(table, box)
    # update memory
    last_box_memory.append(box)
    # remove box that was previously used
    boxes.remove(box)
    # update positions
    boxes_pos = [
        C.frame(box1).getPosition(),
        C.frame(box2).getPosition(),
        C.frame(box3).getPosition(),
    ]
    n_success += 1


del M

n_success: 0
=== placement 0: grasp x place z
place direction: z
1st Block: (box3)
  -- feasible:placement 0: grasp x place z
     { time: 0.271494, evals: 201, done: 1, feasible: 1, sos: 0.0834742, f: 0, ineq: 0.00628501, eq: 0.422923 }
  -- feasible:sub_motion_0--placement 0: grasp x place z
     { time: 0.093137, evals: 22, done: 1, feasible: 1, sos: 1.03642, f: 0, ineq: 9.52361e-07, eq: 0.320169 }
  -- feasible:sub_motion_1--placement 0: grasp x place z
     { time: 0.124189, evals: 18, done: 1, feasible: 1, sos: 0.412627, f: 0, ineq: 0.00137544, eq: 0.326364 }
n_success: 1
=== placement 1: grasp y place z
place direction: z
2nd Block: (box1)
  -- infeasible:placement 1: grasp y place z
     { time: 0.124142, evals: 97, done: 1, feasible: 0, sos: 0.0648748, f: 0, ineq: 0.211929, eq: 0.872721 }
True
n_success: 1
=== placement 1: grasp x place z
place direction: z
2nd Block: (box1)
  -- infeasible:placement 1: grasp x place z
     { time: 0.136215, evals: 121, done: 1, feasible: 0, s

KeyboardInterrupt: 

### On real robot: BotOp

In [9]:
import robotic as ry
import numpy as np
from bridgebuild_env import create_n_boxes
import robot_execution as robex
# this import the local manipulation.py .. to be integrated in robotic..
import manipulation as manip
import random

In [10]:
C = ry.Config()
C.addFile(ry.raiPath("scenarios/pandaSingle.g"))
n_boxes = 3
keep_dist_between = ["box1", "box2", "box3"]
C = create_n_boxes(C=C, N=n_boxes, position="fixed", shelf = True)

# remove so collision with wrist camera is ignored
C.delFrame("panda_collCameraWrist")

# for convenience, a few definitions:
qHome = C.getJointState()
gripper = "l_gripper"
palm = "l_palm"
box1 = "box1"
box2 = "box2"
box3 = "box3"
table = "table"
boxSize = C.frame(box1).getSize()

C.view()

0

In [11]:
C.view()
bot = robex.Robot(C, on_real=True)
bot.goHome(C)

-- bot.cpp:~BotOp:135(0) shutting down BotOp...
-- simulation.cpp:~BotThreadedSim:57(0) shutting down SimThread
-- simulation.cpp:~Simulation:148(0) shutting down Simulation


In [12]:
C.setJointState(qHome)
C.view_raise()
boxes_pos = [
    C.frame(box1).getPosition(),
    C.frame(box2).getPosition(),
    C.frame(box3).getPosition(),
]
boxes = [box1, box2, box3]
boxes_all = [box1, box2, box3]

last_box_memory = []
paths = []
vis = 1
n_success = 0
boxz = [box1, box3, box2]
while n_success <= 2:
    box = np.random.choice(boxes)
    not_picked_boxes = [i for i in boxes_all if i != box]
    # box = boxz[n_success]
    qStart = C.getJointState()
    graspDirection = random.choice(["x", "y"])  # z grasp removed
    placeDirection = "z"  # random.choice(['x', 'y', 'z', 'xNeg', 'yNeg', 'zNeg'])
    info = f"placement {n_success}: grasp {graspDirection} place {placeDirection}"
    print("===", info)

    M = manip.ManipulationModelling(C, info, helpers=[gripper])
    M.setup_pick_and_place_waypoints(
        gripper, box, phases = 3., homing_scale=1.5e-1, accumulated_collisions=True
    )
    M.grasp_box(1., gripper, box, palm, graspDirection)
    print("place direction:", placeDirection)

    # 3rd block constraints
    if n_success > 1:
        print(f"3rd Block: ({box})")
        dir_vec = (
            C.getFrame(last_box_memory[-2]).getPosition()
            - C.getFrame(last_box_memory[-1]).getPosition()
        )
        pos2 = C.getFrame(last_box_memory[-2]).getPosition()
        pos1 = C.getFrame(last_box_memory[-1]).getPosition()
        rel_pos = (pos1 + pos2) / 2 + np.array([0, 0, 0.06])
        dir_vec /= np.linalg.norm(dir_vec)

        M.komo.addObjective([2.0], ry.FS.position, [box], ry.OT.eq, [1e1], rel_pos)
        M.komo.addObjective([2.0], ry.FS.vectorZ, [box], ry.OT.eq, [1e1], dir_vec)

        print("Grasp Direction:", graspDirection)
        """
        if graspDirection == "x":
            M.komo.addObjective(
                [2.0], ry.FS.scalarProductXZ, [box, table], ry.OT.eq, [1e1], [0.0]
            )
        elif graspDirection == "y":
            M.komo.addObjective(
                [2.0], ry.FS.scalarProductYZ, [box, table], ry.OT.eq, [1e1], [0.0]
            )
        
        else:
           M.komo.addObjective([2.], ry.FS.vectorZ, [gripper], ry.OT.eq, [1e1], [0, 0, 1])
        """
    # 2nd block constraints
    if n_success == 1:
        print(f"2nd Block: ({box})")
        M.place_box(2.0, box, table, palm, placeDirection)
        M.komo.addObjective(
            [2.0],
            ry.FS.negDistance,
            [box, last_box_memory[-1]],
            ry.OT.eq,
            [1e1],
            [-0.04],
        )
        M.komo.addObjective(
            [2.0],
            ry.FS.scalarProductXY,
            [box, last_box_memory[-1]],
            ry.OT.eq,
            [1e1],
            [0],
        )
        #M.keep_dist_pairwise([], keep_dist_between, margin=0.08)
        M.keep_distance([], box, last_box_memory[-1], margin = 0.06)
        #M.keep_distance([], box, not_picked_boxes[1], margin = 0.08)

    M.keep_distance([], palm, table)
    
    # 1st block constraints
    if n_success == 0:
        print(f"1st Block: ({box})")
        M.place_box(2.0, box, table, palm, placeDirection)
        # place inital block as chosen by np.random.choice at a random location
        initial_pos = [np.random.uniform(0.0, 0.1), np.random.uniform(0.0, 0.1)]
        print("Trying xy pos: ", initial_pos)
        M.target_relative_xy_position(2.0, box, table, initial_pos)
        M.keep_distance([], box, not_picked_boxes[0], margin = 0.06)
        M.keep_distance([], box, not_picked_boxes[1], margin = 0.06)

    ways = M.solve()

    if not M.feasible:
        print(
            f"Failed placing block number {n_success+1} with grasp direction {graspDirection} at M :("
        )
        continue
    print("Submotion 0")
    M2 = M.sub_motion(0)
    M2.keep_distance([0.3, 0.7], palm, box)

    M2.retract([0.0, 0.2], gripper, dist=0.012)
    M2.approach([0.8, 1.0], gripper)
    
    
    paths.append(M2.solve())

    if not M2.feasible:
        print(
            f"Failed placing block number {n_success+1} with grasp direction {graspDirection} at M2 :("
        )
        continue

    M3 = M.sub_motion(1)

    M3.keep_distance([], table, box)
    if n_success >= 0:
        M3.retract([0.0, 0.2], gripper, dist=0.025)
        M3.approach([0.8, 1.0], gripper)
    

    paths.append(M3.solve())
    if not M3.ret.feasible:
        print(
            f"Failed placing block number {n_success+1} with grasp direction {graspDirection} at M3 :("
        )
        continue

    M_report = M.komo.report(True)

    # M_report = M.fetch_name_error_type(M_report)
    # M.print_name_error_type(M_report)

    M2_report = M2.komo.report(True)
    # M2_report = M.fetch_name_error_type(M2_report)
    # M.print_name_error_type(M2_report)

    M3_report = M3.komo.report(True)
    # M3_report = M.fetch_name_error_type(M3_report)
    # M.print_name_error_type(M3_report)

    if vis == 0:
        M2.play(C)
        C.attach(gripper, box)
        M3.play(C)
        C.attach(table, box)

    bot.execute_path_blocking(C, path=paths[-2])
    C.attach(gripper, box)
    bot.grasp(C)

    bot.execute_path_blocking(C, path=paths[-1])
    C.attach(table, box)
    bot.release(C)

    M_retract = M.sub_motion(2)
    M_retract.retract([0.0, 0.2], gripper, dist=0.1)
    retract_path = M_retract.solve()
    print(f"Retract path:{retract_path}")
    bot.execute_path_blocking(C, path=retract_path)

    # update memory
    last_box_memory.append(box)
    # remove box that was previously used
    boxes.remove(box)
    # update positions
    boxes_pos = [
        C.frame(box1).getPosition(),
        C.frame(box2).getPosition(),
        C.frame(box3).getPosition(),
    ]
    n_success += 1

bot.goHome
del M

=== placement 0: grasp x place z
place direction: z
1st Block: (box2)
Trying xy pos:  [0.06780366254185077, 0.07565401266699041]
  -- feasible:placement 0: grasp x place z
     { time: 0.134147, evals: 73, done: 1, feasible: 1, sos: 0.176201, f: 0, ineq: 5.83119e-05, eq: 0.0537046 }
Submotion 0
  -- feasible:sub_motion_0--placement 0: grasp x place z
     { time: 0.197795, evals: 17, done: 1, feasible: 1, sos: 1.25387, f: 0, ineq: 9.91421e-07, eq: 0.0504977 }
  -- feasible:sub_motion_1--placement 0: grasp x place z
     { time: 0.214083, evals: 21, done: 1, feasible: 1, sos: 0.514143, f: 0, ineq: 0.00141815, eq: 0.0562952 }
-- kin_physx.cpp:addJoint:298(0) ADDING JOINT l_panda_joint7-box2 of type rigid with rel [0, 0, 0]
-- kin_physx.cpp:removeJoint:891(0) REMOVING JOINT 0xa3c8bf0-0x91bd560 of type rigid
  -- feasible:sub_motion_2--placement 0: grasp x place z
     { time: 0.053659, evals: 9, done: 1, feasible: 1, sos: 0.912522, f: 0, ineq: 8.90189e-06, eq: 0.051969 }
Retract path:[[ 0